# Surface data with 10 urbanunits
- This script is used for creating surface data with 10 urban land units for each single point cities;
- do not create 1 LCZ domain type;
- create with complete pct_urban

In [7]:
import xarray as xr
import pandas as pd
import numpy as np
import os

In [9]:
GRIDNAME=['AU-Pre','US-Mi1','US-Wes']
num_city = len(GRIDNAME)
data_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/CESM/bakeup/UrbanPlumber/'
path = data_dir + 'input_files/'
sens_path = data_dir + 'sensitivity_input/'
lcz_type = [0,0,0,0,0,100,0,0,0,0] # modify the 'US-Wes' to be 100% LCZ6

default = {}
for i in range(num_city):
    folder = GRIDNAME[i] + '/'
    filename = 'surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c230710.nc' # the Keith's standard data
    ds = xr.open_dataset(path + folder+ filename)
    default['ds_' + GRIDNAME[i]] = xr.Dataset()
    for var_name in ds.variables:
        variable = ds[var_name]
        if 'numurbl' in variable.dims:
            default['ds_' + GRIDNAME[i]][var_name] = variable
            
            
datasets = {}
for i in range(num_city):
    folder = GRIDNAME[i] + '/'
    filename = 'surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c230710.nc'
    ds = xr.open_dataset(path + folder+ filename)
    datasets['ds_' + GRIDNAME[i]] = xr.Dataset()
    for var_name in ds.variables:
        variable = ds[var_name]
        if 'numurbl' not in variable.dims:
            datasets['ds_' + GRIDNAME[i]][var_name] = variable    
            
numurbl = 10
lsmlat = 1 
lsmlon = 1
numrad = 2
nlevurb = 5            

- The parameter provided by lcz table is: 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'NLEV_IMPROAD', 'T_BUILDING_MIN', 'THICK_ROOF', 'THICK_WALL', 'TK_IMPROAD', 'TK_WALL', 'TK_ROOF'

In [10]:
# c20240702: use local parameters and newly-developed table
# c240929: use all parameters from newly-developed table
var0 = ['WALL_TO_PLAN_AREA_RATIO'] # use the default
var1 = ['ALB_IMPROAD_DIF', 'ALB_IMPROAD_DIR', 'ALB_PERROAD_DIF', 'ALB_PERROAD_DIR', 
        'ALB_ROOF_DIF', 'ALB_ROOF_DIR', 'ALB_WALL_DIF', 'ALB_WALL_DIR']
var2 = ['CANYON_HWR', 'HT_ROOF', 'WIND_HGT_CANYON', 'WTLUNIT_ROOF', 'WTROAD_PERV']   
var3 = ['TK_IMPROAD', 'TK_ROOF', 'TK_WALL', 'CV_ROOF', 'CV_WALL', 'CV_IMPROAD']
var4 = ['NLEV_IMPROAD', 'THICK_ROOF', 'THICK_WALL', 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'T_BUILDING_MIN']
var5 = ['PCT_URBAN']

###change albedo, too high now
ai_cesm = [0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14]
ap_cesm = [0.08, 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08 , 0.08]
ar_cesm = [0.23, 0.28, 0.25, 0.23, 0.23, 0.23, 0.25 , 0.28 , 0.23 , 0.20]
aw_cesm = [0.30, 0.25 , 0.25 , 0.30 , 0.30 , 0.30 , 0.35 , 0.30 , 0.30 , 0.25]
ca_cesm = [2.50, 1.25 , 1.25 , 0.75 , 0.50 , 0.50 , 0.90 , 0.20 , 0.15 , 0.35]
ht_cesm = [37.50 , 17.50 , 6.50 , 37.50 , 17.50 , 6.50 , 3.00 , 6.50 , 6.50 , 10.00]
wi_cesm = np.array(ht_cesm)/2
wtr_cesm = [0.53 , 0.61 , 0.65 , 0.46 , 0.43 , 0.50 , 0.88 , 0.47 , 0.50 , 0.45]
wtp_cesm = [0.10 , 0.20 , 0.33 , 0.50 , 0.43 , 0.43 , 0.60 , 0.25 , 0.82 , 0.60]
li_cesm = [0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8 , 0.8] # TK imperoad
lr_cesm = [1.70 , 1.70 , 1.09 , 1.25 , 1.70 , 1.09 , 1.09 , 1.07 , 1.09 , 2.00] # TK roof: thermal conductivity
lw_cesm = [1.27 , 2.60 , 1.66 , 1.45 , 1.88 , 1.66 , 1.00 , 1.07 , 1.66 , 1.42 ] # TK wall
cr_cesm = [1.32 , 1.32 , 1.32 , 1.80 , 1.32 , 1.32 , 2.00 , 2.11 , 1.32 , 2.00] # CV_ROOF
cw_cesm = [1.54 , 1.54 , 1.54 , 2.00 , 1.54 , 1.54 , 2.00 , 2.11 , 1.54 , 1.59] # CV_WALL
ci_cesm = [1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80 , 1.80] # CV_IMPROAD: volumetric heat capacity
nlev = [3,2,2,2,3,2,2,2,2,2] # NLEV_IMPROAD
zr_cesm = [0.30 , 0.30 , 0.20 , 0.30 , 0.25 , 0.15 , 0.10 , 0.12 , 0.15 , 0.10] # THICK_ROOF
zw_cesm = [0.30 , 0.25 , 0.25 , 0.20 , 0.20 , 0.20 , 0.10 , 0.20 , 0.20 , 0.10] # 'THICK_WALL
ei_cesm = [0.91, 0.91, 0.91, 0.91, 0.91, 0.91, 0.88, 0.91, 0.91, 0.91]
ep_cesm = [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
er_cesm = [0.91, 0.91, 0.91, 0.91, 0.91, 0.91, 0.88, 0.91, 0.91, 0.91] # emissivity
ew_cesm = [0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90]
tmin = [291, 287, 287, 291, 287, 287, 287, 287, 287, 287] # T_BUILDING_MIN
crr_cesm = [x * 1000000 for x in cr_cesm]
cww_cesm = [x * 1000000 for x in cw_cesm]
cii_cesm = [x * 1000000 for x in ci_cesm]

ucp1_cesm = [ai_cesm, ai_cesm, ap_cesm, ap_cesm, 
             ar_cesm, ar_cesm, aw_cesm, aw_cesm] 
ucp2_cesm = [ca_cesm, ht_cesm, wi_cesm, wtr_cesm, wtp_cesm]
ucp3_cesm = [li_cesm, lr_cesm, lw_cesm, crr_cesm, cww_cesm, cii_cesm]
ucp4_cesm = [nlev, zr_cesm, zw_cesm, ei_cesm, ep_cesm, er_cesm, ew_cesm, tmin]

for i in range(num_city):
    for j in var0:
        datasets['ds_' + GRIDNAME[i]][j] = (('numurbl', 'lsmlat', 'lsmlon'), np.zeros((numurbl, lsmlat, lsmlon)))  
    for j in var1:
        datasets['ds_' + GRIDNAME[i]][j] = (('numrad','numurbl', 'lsmlat', 'lsmlon'), np.zeros((numrad, numurbl, lsmlat, lsmlon)))  
    for j in var2:
        datasets['ds_' + GRIDNAME[i]][j] = (('numurbl', 'lsmlat', 'lsmlon'), np.zeros((numurbl, lsmlat, lsmlon)))   
    for j in var3:
        datasets['ds_' + GRIDNAME[i]][j] = (('nlevurb','numurbl', 'lsmlat', 'lsmlon'), np.zeros((nlevurb, numurbl, lsmlat, lsmlon))) 
    for j in var4:
        datasets['ds_' + GRIDNAME[i]][j] = (('numurbl', 'lsmlat', 'lsmlon'), np.zeros((numurbl, lsmlat, lsmlon)))   
    for j in var5:
        datasets['ds_' + GRIDNAME[i]][j] = (('numurbl', 'lsmlat', 'lsmlon'), np.zeros((numurbl, lsmlat, lsmlon)))

In [11]:
# CESM LCZ-table export surface data
for i in range(num_city):
    for j in range(len(var0)):
        for m in range(numurbl):
            datasets['ds_' + GRIDNAME[i]][var0[j]][m, :, :] = default['ds_' + GRIDNAME[i]][var0[j]][2, :, :]
    
    for j in range(len(var1)):
        for m in range(numurbl):
            datasets['ds_' + GRIDNAME[i]][var1[j]][:, m, :, :] = ucp1_cesm[j][m]

    for j in range(len(var2)):
        for m in range(numurbl):
            datasets['ds_' + GRIDNAME[i]][var2[j]][m, :, :] = ucp2_cesm[j][m]

    for j in range(len(var3)):
        for m in range(numurbl):
            datasets['ds_' + GRIDNAME[i]][var3[j]][:, m, :, :] = ucp3_cesm[j][m]

    for j in range(len(var4)):
        for m in range(numurbl):
            datasets['ds_' + GRIDNAME[i]][var4[j]][m, :, :] = ucp4_cesm[j][m]
    
    for m in range(numurbl):
        datasets['ds_' + GRIDNAME[i]][var5[0]][m, :, :] = lcz_type[m]
            
for i in range(num_city):
    output = sens_path + GRIDNAME[i] +'/surfdata_1x1_'+GRIDNAME[i]+'_detailed_simyr2000_c240929base.nc'
    if os.path.exists(output):
        os.remove(output)
    datasets['ds_' + GRIDNAME[i]].to_netcdf(output)                

# Description

In [17]:
check1= xr.open_dataset(path + GRIDNAME[9] +'/surfdata_1x1_'+GRIDNAME[9]+'_detailed_simyr2000_c240929cesm.nc')
check1['TK_ROOF'].values # check if 78 variables

array([[[[1.7 ]],

        [[1.7 ]],

        [[1.09]],

        [[1.25]],

        [[1.7 ]],

        [[1.09]],

        [[1.09]],

        [[1.07]],

        [[1.09]],

        [[2.  ]]],


       [[[1.7 ]],

        [[1.7 ]],

        [[1.09]],

        [[1.25]],

        [[1.7 ]],

        [[1.09]],

        [[1.09]],

        [[1.07]],

        [[1.09]],

        [[2.  ]]],


       [[[1.7 ]],

        [[1.7 ]],

        [[1.09]],

        [[1.25]],

        [[1.7 ]],

        [[1.09]],

        [[1.09]],

        [[1.07]],

        [[1.09]],

        [[2.  ]]],


       [[[1.7 ]],

        [[1.7 ]],

        [[1.09]],

        [[1.25]],

        [[1.7 ]],

        [[1.09]],

        [[1.09]],

        [[1.07]],

        [[1.09]],

        [[2.  ]]],


       [[[1.7 ]],

        [[1.7 ]],

        [[1.09]],

        [[1.25]],

        [[1.7 ]],

        [[1.09]],

        [[1.09]],

        [[1.07]],

        [[1.09]],

        [[2.  ]]]])

In [27]:
check2 = xr.open_dataset(path + GRIDNAME[9] +'/surfdata_1x1_'+GRIDNAME[9]+'_detailed_simyr2000_c240322li.nc')
check2['THICK_WALL'][1].values

array([[0.25]])

# Desciption 
- tuning the T_BUILD_MIN according to surface data

In [30]:
def_surf = xr.open_dataset('/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/Projects/inputdata/lnd/clm2/surfdata_esmf/ctsm5.2.0/surfdata_0.9x1.25_hist_78pfts_CMIP6_1850_c230517.nc')

In [40]:
urban_mask = def_surf['PCT_URBAN'] > 0
urban_mask

<xarray.DataArray 'PCT_URBAN' (numurbl: 3, lsmlat: 192, lsmlon: 288)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Dimensions without coordinates: numurbl, lsmlat, lsmlon

In [53]:
var = def_surf['CANYON_HWR'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

5.05 2.4 8.0
1.56 0.8 1.8
0.65 0.32 1.6


In [54]:
var= def_surf['HT_ROOF'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

126.34 60.0 200.0
39.02 20.0 45.0
13.17 8.0 17.0


In [55]:
var= def_surf['THICK_ROOF'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2)) 

0.25 0.12 0.26
0.2 0.12 0.26
0.18 0.12 0.26


In [57]:
var= def_surf['THICK_WALL'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.32 0.2 0.32
0.3 0.2 0.32
0.29 0.19 0.32


In [139]:
var= def_surf['WTLUNIT_ROOF'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.61 0.4 0.85
0.61 0.4 0.8
0.44 0.2 0.8


In [65]:
var= def_surf['WTROAD_PERV'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.27 0.12 0.71
0.42 0.25 1.0
0.68 0.43 1.0


In [67]:
var= def_surf['ALB_ROOF_DIR'].where(urban_mask)
for i in range(3):
    print(np.round(var[0,i].mean().values,2), np.round(var[0,i].min().values,2), np.round(var[0,i].max().values,2))

0.15 0.14 0.61
0.24 0.14 0.61
0.24 0.14 0.61


In [138]:
var= def_surf['ALB_WALL_DIR'].where(urban_mask)
for i in range(3):
    print(np.round(var[0,i].mean().values,2), np.round(var[0,i].min().values,2), np.round(var[0,i].max().values,2))

0.22 0.22 0.55
0.25 0.22 0.55
0.27 0.22 0.55


In [69]:
var= def_surf['ALB_IMPROAD_DIR'].where(urban_mask)
for i in range(3):
    print(np.round(var[0,i].mean().values,2), np.round(var[0,i].min().values,2), np.round(var[0,i].max().values,2))

0.22 0.13 0.23
0.16 0.13 0.72
0.23 0.08 0.72


In [140]:
var= def_surf['ALB_PERROAD_DIR'].where(urban_mask)
for i in range(3):
    print(np.round(var[0,i].mean().values,2), np.round(var[0,i].min().values,2), np.round(var[0,i].max().values,2))

0.08 0.08 0.08
0.08 0.08 0.08
0.08 0.08 0.08


In [87]:
var= def_surf['EM_ROOF'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.89 0.04 0.91
0.86 0.04 0.92
0.87 0.04 0.92


In [88]:
var= def_surf['EM_WALL'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.9 0.8 0.91
0.9 0.8 0.91
0.9 0.8 0.93


In [89]:
var= def_surf['EM_IMPROAD'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.88 0.88 0.91
0.89 0.28 0.91
0.8 0.28 0.95


In [90]:
var= def_surf['EM_PERROAD'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values,2), np.round(var[i].min().values,2), np.round(var[i].max().values,2))

0.95 0.95 0.95
0.95 0.95 0.95
0.95 0.95 0.95


In [101]:
var= def_surf['TK_ROOF'].where(urban_mask)
for i in range(3):
    print(np.round(var[:,i].mean().values,2), np.round(var[:,i].min().values,2), np.round(var[:,i].max().values,2))

0.65 0.03 45.0
0.82 0.03 45.0
0.68 0.03 45.0


In [144]:
var= def_surf['TK_WALL'].where(urban_mask)
for i in range(3):
    #print(np.round(var[:,i].mean().values,2), np.round(var[:,i].min().values,2), np.round(var[:,i].max().values,2))
    print(np.round(var.sum(axis=0)[i].mean().values,2), np.round(var.mean(axis=0)[i].min().values,2), np.round(var.mean(axis=0)[i].max().values,2))

0.07 0.83 3.37
3.03 0.83 9.23
3.64 0.83 14.3


In [109]:
var= def_surf['TK_IMPROAD'].where(urban_mask)
for i in range(3):
    print(np.round(var[:,i].mean().values,2), np.round(var[:,i].min().values,2), np.round(var[:,i].max().values,2))

0.28 0.0 1.9
0.22 0.0 1.9
0.15 0.0 1.67


In [111]:
var= def_surf['CV_ROOF'].where(urban_mask)
for i in range(3):
    print(np.round(var[:,i].mean().values/1000000,2), np.round(var[:,i].min().values/1000000,2), np.round(var[:,i].max().values/1000000,2))

0.71 0.0 3.74
0.58 0.0 3.74
0.53 0.0 3.74


In [112]:
var= def_surf['CV_WALL'].where(urban_mask)
for i in range(3):
    print(np.round(var[:,i].mean().values/1000000,2), np.round(var[:,i].min().values/1000000,2), np.round(var[:,i].max().values/1000000,2))

0.96 0.1 2.17
0.91 0.1 2.18
0.89 0.1 2.18


In [113]:
var= def_surf['CV_IMPROAD'].where(urban_mask)
for i in range(3):
    print(np.round(var[:,i].mean().values/1000000,2), np.round(var[:,i].min().values/1000000,2), np.round(var[:,i].max().values/1000000,2))

0.53 0.0 2.1
0.4 0.0 2.1
0.33 0.0 2.06


In [84]:
var= def_surf['T_BUILDING_MIN'].where(urban_mask)
for i in range(3):
    print(np.round(var[i].mean().values-273.15,2), np.round(var[i].min().values-273.15,2), np.round(var[i].max().values-273.15,2))

17.87 16.95 18.95
13.15 11.95 18.95
12.94 11.95 16.95


In [38]:
def_surf['CV_WALL'].where(urban_mask)[:,0,:,:].mean()

<xarray.DataArray 'CV_WALL' ()>
array(961933.06564116)